In [1]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 42 not upgraded.
Need to get 88.4 MB of archives.
After this operation, 297 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 100.0.4896.127-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser a

In [2]:
from selenium import webdriver
options = webdriver.ChromeOptions()

In [3]:
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [4]:
driver = webdriver.Chrome(options=options)

In [5]:
driver.maximize_window()

In [6]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By

### Reading Link File

In [7]:
links = pd.read_excel('Input.xlsx')
links.head()

,URL_ID,URL
0,1.0,https://insights.blackcoffer.com/how-is-login-...
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...
3,4.0,https://insights.blackcoffer.com/how-do-deep-l...
4,5.0,https://insights.blackcoffer.com/how-artificia...


In [9]:
for  index,row in links.head(2).iterrows():
  print(row['URL_ID'],row['URL'])

1.0 https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/
2.0 https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/


### Now we will scrape data from the data given in these links

In [10]:
# function to scrape data from the links
def scrape_data(link):
  global driver
  driver.get(link)
  title=driver.find_element(By.XPATH,"//div[contains(@class,'td-post-content')]")
  driver.implicitly_wait(10)
  return title.text

In [16]:
# function to save the scraped files
def save_file(scrapdata):
  for data in scrapdata:
    fname=str(data['URL-ID'])+".txt"
    f=open("./sample_data/scraped_files/"+fname,'w+',encoding='utf-8')
    f.write(data['TEXT'])
    f.close()

### Perfroming Scraping operation

In [18]:
data=[]
for index,row in links.iterrows():
  item={}
  item['URL-ID']=row['URL_ID']
  item['TEXT']=scrape_data(row['URL'])
  data.append(item)
save_file(data)
       

### Making a data frame of scrapped data

In [19]:
df=pd.DataFrame(data)

In [20]:
df.head(5)

,URL-ID,TEXT
0,1.0,When people hear AI they often think about sen...
1,2.0,"With increasing computing power and more data,..."
2,3.0,If you were a fan of the 90’s film Clueless ba...
3,4.0,"Understanding exactly how data is ingested, an..."
4,5.0,"From the stone age to the modern world, from h..."


In [22]:
# Save all data in one csv file
df.to_csv('contect.csv',index=None)

### let us do some pre processing of the data before we perform sentiment  analysis on it

In [23]:
df["Number of sentences"]=df['TEXT'].apply(lambda x:len(x.split('.')))

In [24]:
df.head(5)

,URL-ID,TEXT,Number of sentences
0,1.0,When people hear AI they often think about sen...,28
1,2.0,"With increasing computing power and more data,...",29
2,3.0,If you were a fan of the 90’s film Clueless ba...,77
3,4.0,"Understanding exactly how data is ingested, an...",15
4,5.0,"From the stone age to the modern world, from h...",60


### Replacing short form of words

In [25]:
def short_forms():    
    return {
        "cant":"can not",
        "dont":"do not",
        "wont":"will not",
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks",
        "couldn't":"could not",
        "wouldn't":"would not",
        "shouldn't":"should not",
        "im":"i am"
        }

In [26]:
import re  ##check if a particular string matches a given regular expression
import string

## funtion to replace the short forms 
def normalization(data):
    data = str(data).lower()
    # URL
    data = re.sub('((www.[^\s]+)|(https?://[^\s]+))',' ',data)
    data = re.sub(r'#([^\s]+)', r'\1', data)

    # Number
    data = ''.join([i for i in data if not i.isdigit()])

    # Punctuation

    for sym in string.punctuation:
        data = data.replace(sym, " ")
    short_form = short_forms()
    data = data.replace("’","'")
    words = data.split()
    converted = [short_form[word] if word in short_form else word for word in words]
    data = " ".join(converted)
    return data

In [27]:
df['TEXT']=df['TEXT'].apply(normalization)

In [29]:
df['TEXT']=df['TEXT'].apply(lambda x:x.lower())

In [30]:
df.head()

,URL-ID,TEXT,Number of sentences
0,1.0,when people hear ai they often think about sen...,28
1,2.0,with increasing computing power and more data ...,29
2,3.0,if you were a fan of the 's film clueless back...,77
3,4.0,understanding exactly how data is ingested ana...,15
4,5.0,from the stone age to the modern world from hu...,60


### Performing Sentiment Analysis

In [31]:
# LoughranMcDonald_MasterDictionary_2020 is a dictionary which contains the 
#sentiment analysis words which will act as a reference for our data set words

guide=pd.read_csv('LoughranMcDonald_MasterDictionary_2020.csv')
guide.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,312,1.422050e-08,1.335201e-08,3.700747e-06,96,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.367356e-10,8.882163e-12,9.362849e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,4.102067e-10,1.200533e-10,5.359747e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,15,6.836779e-10,4.080549e-10,1.406914e-07,14,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8009,3.650384e-07,3.798698e-07,3.523914e-05,1058,0,0,0,0,0,0,0,0,3,12of12inf


In [54]:
guide[guide['Negative']>0]['Word']

9             ABANDON
10          ABANDONED
11         ABANDONING
12        ABANDONMENT
13       ABANDONMENTS
             ...     
86080      WRONGDOING
86081     WRONGDOINGS
86085        WRONGFUL
86086      WRONGFULLY
86094         WRONGLY
Name: Word, Length: 2345, dtype: object

### Assigning Positive and Negative score to our words based on the dictionary words

In [66]:
pos = [] 
neg =[]
Uncertain = []
for index,row in guide.iterrows():
    if row['Negative']>0:
        neg.append(row['Word'].lower())
    elif row['Positive']>0:
        pos.append(row['Word'].lower())
    elif row['Uncertainty']>0:
        Uncertain.append(row['Word'].lower())

In [56]:
df.head()

,URL-ID,TEXT,Number of sentences,PositiveScore,NegativeScore,POLARITY SCORE,WORD COUNT,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,AVG WORD LENGTH,PERSONAL PRONOUNS
0,1.0,when people hear ai they often think about sen...,28,0,0,0.0,724,0.0,25.857143,25.857143,5.059392,4
1,2.0,with increasing computing power and more data ...,29,0,0,0.0,639,0.0,22.034483,22.034483,5.250391,2
2,3.0,if you were a fan of the 's film clueless back...,77,0,0,0.0,1852,0.0,24.051948,24.051948,5.210043,13
3,4.0,understanding exactly how data is ingested ana...,15,0,0,0.0,434,0.0,28.933333,28.933333,5.301843,1
4,5.0,from the stone age to the modern world from hu...,60,0,0,0.0,1249,0.0,20.816667,20.816667,4.916733,28


In [57]:
def positivescore(text):
  score=0
  global pos
  words=text.split()
  for word in words:
    if word in pos:
      score+=1
  return score

In [58]:
def negativescore(text):
  score=0
  global neg
  words=text.split()
  for word in words:
    if word in neg:
      score -= 1
  return score

In [67]:
df['PositiveScore']=df['TEXT'].apply(positivescore)
df['NegativeScore']=df['TEXT'].apply(negativescore)

### Getting all the different parameters

In [68]:
df['POLARITY SCORE']=(df['PositiveScore']-df['NegativeScore'])/ ((df['PositiveScore'] + df['NegativeScore']) + 0.000001)
df['WORD COUNT']=df['TEXT'].apply(lambda x:len(x.split()))
df['SUBJECTIVITY SCORE']=(df['PositiveScore'] + df['NegativeScore'])/ ((df['WORD COUNT']) + 0.000001)
df['AVG SENTENCE LENGTH']=df['WORD COUNT']/df['Number of sentences']
df['AVG NUMBER OF WORDS PER SENTENCE'] = df['WORD COUNT']/df['Number of sentences']

In [61]:
## for avg length of words
def avgwordlength(text):
    words = text.split()
    no_of_words=len(words)
    total_char=0
    for word in words:
        total_char+=len(word)
    return total_char/no_of_words

In [62]:
## for seeing if the sentence has pronoun
def pronoun(text):
    pronouns = r"(\b(s?i|me|we|my|ours|us|I|Me|We|My|Ours|Us)\b)"
    result = 0

    matches = re.finditer(pronouns,text,re.MULTILINE)
    for nummatch,match in enumerate(matches):
        result+=1
    return result

In [63]:
df['AVG WORD LENGTH']=df['TEXT'].apply(avgwordlength)
df['AVG SENTENCE LENGTH']=df['WORD COUNT']/df['Number of sentences']
df['PERSONAL PRONOUNS']=df['TEXT'].apply(pronoun)

In [69]:
df[df['PositiveScore']>0]

,URL-ID,TEXT,Number of sentences,PositiveScore,NegativeScore,POLARITY SCORE,WORD COUNT,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,AVG WORD LENGTH,PERSONAL PRONOUNS
0,1.0,when people hear ai they often think about sen...,28,4,-7,-3.666668,724,-0.004144,25.857143,25.857143,5.059392,4
1,2.0,with increasing computing power and more data ...,29,8,-6,6.999997,639,0.003130,22.034483,22.034483,5.250391,2
2,3.0,if you were a fan of the 's film clueless back...,77,34,-19,3.533333,1852,0.008099,24.051948,24.051948,5.210043,13
3,4.0,understanding exactly how data is ingested ana...,15,5,-1,1.500000,434,0.009217,28.933333,28.933333,5.301843,1
4,5.0,from the stone age to the modern world from hu...,60,21,-17,9.499998,1249,0.003203,20.816667,20.816667,4.916733,28
...,...,...,...,...,...,...,...,...,...,...,...,...
165,167.0,can academia researchers decision makers and p...,67,15,-40,-2.200000,1520,-0.016447,22.686567,22.686567,5.061184,15
166,168.0,introduction inventory planning is a fundament...,43,21,-12,3.666666,962,0.009356,22.372093,22.372093,5.460499,0
167,169.0,the problem insider threat detection specifica...,63,3,-47,-1.136364,975,-0.045128,15.476190,15.476190,5.355897,6
168,170.0,if we talk in terms of our general life exfilt...,31,4,-10,-2.333334,588,-0.010204,18.967742,18.967742,4.833333,11


In [73]:
df['URL']=links['URL']

In [74]:
df.head()

,URL-ID,TEXT,Number of sentences,PositiveScore,NegativeScore,POLARITY SCORE,WORD COUNT,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,AVG WORD LENGTH,PERSONAL PRONOUNS,URL
0,1.0,when people hear ai they often think about sen...,28,4,-7,-3.666668,724,-0.004144,25.857143,25.857143,5.059392,4,https://insights.blackcoffer.com/how-is-login-...
1,2.0,with increasing computing power and more data ...,29,8,-6,6.999997,639,0.003130,22.034483,22.034483,5.250391,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3.0,if you were a fan of the 's film clueless back...,77,34,-19,3.533333,1852,0.008099,24.051948,24.051948,5.210043,13,https://insights.blackcoffer.com/ai-and-its-im...
3,4.0,understanding exactly how data is ingested ana...,15,5,-1,1.500000,434,0.009217,28.933333,28.933333,5.301843,1,https://insights.blackcoffer.com/how-do-deep-l...
4,5.0,from the stone age to the modern world from hu...,60,21,-17,9.499998,1249,0.003203,20.816667,20.816667,4.916733,28,https://insights.blackcoffer.com/how-artificia...
